In [32]:
import os
import json
import pymongo
import musicbrainzngs
import arrow
import re
import time
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict, OrderedDict

from dateutil import parser
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


In [103]:
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
# setup mongo
client = MongoClient()
db = client["bandstour"]
venues =  db["venues"]
artists =  db["selectedArtists"]
FUZZ_RATIO = 90 
NB_remplacements = 0


#
gigsCollection = db['bandsintown']
selectedGigsCollection = db['selectedGigs']
venuesCollection = db['venues']
#

#
def sampleGigs(minShowsPerVenue, db, gigsCollection, venuesCollection): 
    
    data = getSelectedVenues(minShowsPerVenue, venuesCollection)
    
    cloneGigsInSelectedVenues(data["selectedVenues"], gigsCollection, "selectedGigs")
    storeAllArtists(selectedGigsCollection, "selectedArtists")
    storeAllVenues(selectedGigsCollection, "selectedVenues")


def cloneGigsInSelectedVenues(selectedVenues, gigsCollection, newCollName) :
    print len(selectedVenues)
    start = time.strftime("%c")
    
    result = gigsCollection.aggregate([{ "$match": { "venue.place" : { "$in" : selectedVenues } } },{ "$out": newCollName }])
    
    end = time.strftime("%c")
    print "Artists execution time: %dms",start, end
    print result
    #reo=gigsCollection.createIndex([("artists" , pymongo.ASCENDING)], unique=True)
    #rep=gigsCollection.createIndex([("artists" , pymongo.DESCENDING)], unique=True)
    #TODO:RECREATE INDEXES FROM HERE
    #gigsCollection.ensure_index( {'artists':1 } )
    #gigsCollection.ensure_index( {'artists':-1 } )
    #return result[0]

def getSelectedVenues(minShows, venuesCollection) :
    start = time.strftime("%c")
    
    resultV= venuesCollection.aggregate([{"$project": {  "id" : 1,"count" : 1 }}
    , { "$match" : {"count" : { "$gte"  :  minShows }} }
    , { "$group": {"_id": "_id", "selectedVenues": { "$push": "$_id" }}}]
    ,allowDiskUse = True)

    end = time.strftime("%c") 
    print ("Execution time: %dms",start, end)
    resultVenue =[]
    for res in resultV:
        
        #print res
        resultVenue.append(res)
    #print resultVenue
    return resultVenue[0]
    
    
    
def getGigsByArtist(artist, gigsCollection):
    start = time.strftime("%c")

    resultA= gigsCollection.aggregate([{"$project": {
                "artists" : 1,
                "venue" : 1 ,
                "datetime" : 1,
                "y" : { "$year": "$datetime" },
                "m" : { "$month": "$datetime" },
                "d" : { "$dayOfMonth": "$datetime" }

            }
        }
        , { "$unwind": "$artists" } # développer array pour pouvoir en lire les valeurs
        , { "$match" : { "artists.name" : artist} }
        , { "$group": {
            "_id": "$artists.name"
            , "gigs": { "$push": { "venue" : "$venue", "year" : "$y", "month" : "$m", "day" : "$d", "datetime" : "$datetime" } }\
            }
        }
        ],allowDiskUse = True)
#
    end = time.strftime("%c") 
    print "GET single artist execution time: %dms",start, end
    resultArtist =[]
    for resA in resultA:
        
        #print res
        resultArtist.append(resA)
    return resultArtist[0]
                         
                         
def storeAllVenues (collection, newCollName):
#
    start = time.strftime("%c")
#
    collection.aggregate([
        {
            "$project": { # select fields
                "venue" : 1
            }
        }

        ,{
            "$group": {
                "_id":  "$venue.place",  # gather artists name
                 "city" : { "$first" : "$venue.city"}
                , "name" : { "$first" : "$venue.name"}
                #, "url" : { "$first" : "$venue.url"}
                , "country" : { "$first" : "$venue.country"}
                , "region" : { "$first" : "$venue.region"}
                , "longitude" : { "$first" : "$venue.longitude"}
                , "latitude" : { "$first" : "$venue.latitude"}
                # , "id" : { "$first" : "id"}

                , "count" : { "$sum" :  1 }# count the number of artists
            }
        }
        , { "$out" : newCollName}
    ]
    ,allowDiskUse = True # end pipeline
    )#
    end = time.strftime("%c")
    print "Venues execution time: %dms", start,end

#
def storeAllArtists (collection, newCollName):
#
    start = time.strftime("%c")
#
    collection.aggregate([
        {
            "$project": { # select fields
                "artists" : 1,
                "venue" : 1,
                "datetime" : 1
            }
        }
        , { "$unwind": "$artists" } # développer array pour pouvoir en lire les valeurs
        ,{
            "$group": {
                "_id":  "$artists.name"  # gather artists name
                , "mbid" : { "$first" : "$artists.mbid"}
                , "count" : { "$sum" :  1 }# count the number of artists
                , "gigs": {
                    "$push": {
                        "venue" : "$venue",
                        "datetime" : "$datetime"
                    }
                }

            }
        }
        , { "$out" : newCollName}
    ] # end pipeline
    ,allowDiskUse = True)
#
    end = time.strftime("%c")
    print "Artists execution time: %dms",start, end

    
    
        

In [105]:
#HERE STARTS THE FUN!
storeAllVenues(gigsCollection, "venues")

sampleGigs(2, db, gigsCollection, venuesCollection)
    

Venues execution time: %dms Sun Jun 11 02:54:51 2017 Sun Jun 11 02:55:11 2017
('Execution time: %dms', 'Sun Jun 11 02:55:11 2017', 'Sun Jun 11 02:55:12 2017')
184440
Artists execution time: %dms Sun Jun 11 02:55:12 2017 Sun Jun 11 02:55:58 2017
Artists execution time: %dms Sun Jun 11 02:55:58 2017 Sun Jun 11 02:56:30 2017
Venues execution time: %dms Sun Jun 11 02:56:30 2017 Sun Jun 11 02:56:42 2017
